In [9]:
# import libraries
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.optimize as sco

In [10]:
# Define the list of 15 stocks selected from the unsupervised stage.
stocks = ["ORLY","CPRT","AAPL","DHR","TDG","COST","WM","AJG","CTAS","MCD","BKNG","CMI","AMP","DFS","REGN"]

# Initialize dictionaries to store predictions, MSE, and loss histories
predicted_prices = {}
mse_dict = {}
mape_dict = {}
loss_history = {}

# Define parameters for LSTM
seq_length = 60
epochs = 10
batch_size = 32

# Function to create sequences training and test data
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(data[i + seq_length])
    return np.array(sequences), np.array(targets)

# Train LSTM model and predict for each stock
for ticker in stocks:
    data = yf.download(ticker, start="2019-01-01", end="2023-12-31")
    data = data[['Close']]

    scaler = MinMaxScaler(feature_range=(0, 1))
    data_scaled = scaler.fit_transform(data)

    X, y = create_sequences(data_scaled, seq_length)

    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Reshape data for LSTM: LSTM require data to be in 3D format: (samples, time steps, features)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    # Build the LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        LSTM(50, return_sequences=False),
        Dropout(0.2),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    # Predict the test set
    predicted = model.predict(X_test)
    predicted_stock_price = scaler.inverse_transform(predicted)

    # Store the predictions
    test_dates = data.index[-len(predicted_stock_price):]
    predicted_prices[ticker] = pd.Series(predicted_stock_price.flatten(), index=test_dates)

    # Calculate and store MSE
    actual_stock_price = scaler.inverse_transform(y_test.reshape(-1, 1))
    mse = mean_squared_error(actual_stock_price, predicted_stock_price)
    mse_dict[ticker] = mse

    # Calculate the MAPE
    mape = np.mean(np.abs((actual_stock_price - predicted_stock_price) / predicted_stock_price)) * 100
    mape_dict[ticker] = mape

    # Store loss history
    loss_history[ticker] = history.history

# convert MSE and MAPE into dataframe
mse_df = pd.DataFrame(list(mse_dict.items()), columns=['Stock', 'MSE'])
mse_df['RSME'] = np.sqrt(mse_df['MSE'])
mape_df = pd.DataFrame(list(mape_dict.items()), columns=['Stock', 'MAPE'])

# Combine all predictions into a DataFrame
# This dataframe contains the predicted price of all 15 stocks.
predicted_df = pd.DataFrame(predicted_prices)



[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step


[*********************100%%**********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step


In [11]:
# Plot the loss for each stock
fig = go.Figure()

for ticker in stocks:
    # Add training loss
    fig.add_trace(go.Scatter(
        x=list(range(1, epochs + 1)),
        y=loss_history[ticker]['loss'],
        mode='lines',
        name=f'{ticker} Training Loss'
    ))

# Update layout
fig.update_layout(
    title="Training Loss for 15 Stocks",
    xaxis_title="Epochs",
    yaxis_title="Loss (MSE)",
    showlegend=True
)

# Show the plot
fig.show()

# Create a subplot figure with 5 rows and 3 columns
fig = make_subplots(rows=5, cols=3, subplot_titles=stocks, vertical_spacing=0.1, horizontal_spacing=0.1)

# Add traces for each stock
for i, ticker in enumerate(stocks):
    row = i // 3 + 1
    col = i % 3 + 1

    # Fetch the actual test set prices for each stock
    data = yf.download(ticker, start="2019-01-01", end="2023-12-31")
    data = data[['Close']]
    data_scaled = scaler.fit_transform(data)
    _, y = create_sequences(data_scaled, seq_length)
    y_train, y_test = y[:train_size], y[train_size:]
    actual_stock_price = scaler.inverse_transform(y_test.reshape(-1, 1))

    fig.add_trace(go.Scatter(x=test_dates, y=actual_stock_price.flatten(), mode='lines', name=f'{ticker} Actual (Black)', line=dict(color='black')), row=row, col=col)
    fig.add_trace(go.Scatter(x=test_dates, y=predicted_prices[ticker], mode='lines', name=f'{ticker} Predicted (Blue)', line=dict(color='royalblue')), row=row, col=col)

# Update layout
fig.update_layout(
    height=1200,  # Adjust the height as needed
    width=1200,  # Adjust the width as needed
    title_text="Actual vs Predicted Stock Prices for 15 Stocks",
    showlegend=False,
    template="plotly_white"
)

# Show the plot
fig.show()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [12]:
# Calculate daily returns from the predicted prices
predicted_returns = predicted_df.pct_change().dropna()

# Function to calculate portfolio performance
def portfolio_performance(weights, returns):
    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    return portfolio_return, portfolio_volatility

# Objective function to minimize
def negative_sharpe_ratio(weights, returns, risk_free_rate=0.01):
    p_return, p_volatility = portfolio_performance(weights, returns)
    return -(p_return - risk_free_rate) / p_volatility

# Constraints and bounds
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0, 1) for _ in range(len(stocks)))

# Initial guess - we start of with an initial equal weight (1/N) N: number of stocks
initial_weights = np.array([1./len(stocks)] * len(stocks))

# Optimize portfolio using predicted returns

optimal_results_predicted = sco.minimize(negative_sharpe_ratio, initial_weights, args=(predicted_returns,), method='SLSQP', bounds=bounds, constraints=constraints)
optimal_weights_predicted = optimal_results_predicted.x

# Create Equal-weighted portfolio
equal_weights = np.array([1./len(stocks)] * len(stocks))

# Fetch actual data from 2024-01-01 to 2024-07-31
actual_data = yf.download(stocks, start="2024-01-01", end="2024-07-31")['Close']

# Calculate actual daily returns
actual_returns = actual_data.pct_change().dropna()

# Calculate cumulative returns of the equal-weighted portfolio
equal_portfolio_daily_returns = (actual_returns * equal_weights).sum(axis=1)
equal_cumulative_returns = (1 + equal_portfolio_daily_returns).cumprod()

# Calculate cumulative returns of the predicted portfolio
predicted_portfolio_daily_returns = (actual_returns * optimal_weights_predicted).sum(axis=1)
predicted_cumulative_returns = (1 + predicted_portfolio_daily_returns).cumprod()

# Plot cumulative returns
fig = go.Figure()
fig.add_trace(go.Scatter(x=predicted_cumulative_returns.index, y=predicted_cumulative_returns.values, mode='lines', name='LSTM Optimized Portfolio'))
fig.add_trace(go.Scatter(x=equal_cumulative_returns.index, y=equal_cumulative_returns.values, mode='lines', name='Equal-Weighted Portfolio'))

fig.update_layout(title='Cumulative Returns Comparison (2024-01-01 to 2024-07-31)',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return',
                  xaxis=dict(tickformat='%Y-%m-%d'),
                  template="plotly_white",
                  )

fig.show()


[*********************100%%**********************]  15 of 15 completed


In [13]:
# Download S&P 500 data
sp500_data = yf.download("^GSPC", start="2024-01-01", end="2024-07-31")['Close']

# Calculate S&P 500 returns
sp500_returns = sp500_data.pct_change().dropna()

# Calculate cumulative returns for S&P 500 for 2024-01-01 ~ 2024-07-31.
sp500_cumulative_returns = (1 + sp500_returns).cumprod()

# Plot cumulative returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=predicted_cumulative_returns.index, y=predicted_cumulative_returns.values, mode='lines', name='Predicted Optimized Portfolio'))
fig.add_trace(go.Scatter(x=sp500_cumulative_returns.index, y=sp500_cumulative_returns.values, mode='lines', name='S&P 500 Index'))

fig.update_layout(title='Cumulative Returns Comparison (2024-01-01 to 2024-07-31) S&P 500 vs Optimized Portfolio',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return',
                  xaxis=dict(tickformat='%Y-%m-%d'),
                  template="plotly_white")

fig.show()

[*********************100%%**********************]  1 of 1 completed


In [14]:
# Generate random portfolios
num_portfolios = 5000
all_weights = np.zeros((num_portfolios, len(stocks)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

for i in range(num_portfolios):
    # Generate random weights
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)

    # Save weights
    all_weights[i,:] = weights

    # Expected return and volatility
    ret_arr[i], vol_arr[i] = portfolio_performance(weights, predicted_returns)
    sharpe_arr[i] = ret_arr[i] / vol_arr[i]

# Plotting the efficient frontier
fig = go.Figure()

# Plot random portfolios
fig.add_trace(go.Scatter(x=vol_arr, y=ret_arr, mode='markers',
                         marker=dict(color=sharpe_arr, colorscale='Viridis', size=5, colorbar=dict(title='Sharpe Ratio')),
                         name='Portfolios'))

# Plot optimized portfolios
pred_return, pred_volatility = portfolio_performance(optimal_weights_predicted, predicted_returns)

fig.add_trace(go.Scatter(x=[pred_volatility], y=[pred_return], mode='markers+text', text=["Predicted Optimal"],
                         textposition='bottom center', marker=dict(color='red', size=10), name='Predicted Optimal'))


fig.update_layout(title='Efficient Frontier',
                  xaxis_title='Volatility',
                  yaxis_title='Return',
                  showlegend=False)

fig.show()